In [1]:
import openai
from openai import OpenAI
import os
import base64
import requests
from key import OPENAI_API_KEY

api_key=OPENAI_API_KEY
client = OpenAI(api_key=api_key)

In [2]:
# Function to encode the image
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

def list_files(directory):
    file_list = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            file_list.append(os.path.join(root, file))
    return file_list

In [3]:
# Get System Prompt
f = open("prompt.txt", "r")
prompt_text = f.read()

# Get User Input
user_input = "1. Dunks 2. Crossovers"

In [4]:
# Get all the image paths
directory = './frames'  # When you run video.py, all the frames located inside frames folder
image_paths = list_files(directory)

results = []

# Iterate through each image path
for image_path in image_paths:
    # Getting the base64 string
    base64_image = encode_image(image_path)

    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }

    payload = {
        "model": "gpt-4o",
        "messages": [
            {"role": "system", "content": prompt_text},
            {"role": "user", "content": [{"type": "text", "text": user_input}, {"type": "image_url","image_url": {"url": f"data:image/jpeg;base64,{base64_image}", "detail": "low"}}]}
            # user_input text can be removed
        ],
        "max_tokens": 300
    }

    # Make the API request
    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

    # Append the result to the list
    results.append(response.json())

In [5]:
# Result Sample
results[0]

{'id': 'chatcmpl-9cy7XZE3AGh4Rr4LhmP3AQWOnZ8QD',
 'object': 'chat.completion',
 'created': 1719074595,
 'model': 'gpt-4o-2024-05-13',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': 'Player in white jersey driving towards the basket past a defender in a teal jersey'},
   'logprobs': None,
   'finish_reason': 'stop'}],
 'usage': {'prompt_tokens': 208, 'completion_tokens': 15, 'total_tokens': 223},
 'system_fingerprint': 'fp_9cb5d38cf7'}

In [6]:
# Iterate through all the results
for i in range(len(image_paths)):
    print(results[i]['choices'][0]['message']['content'])

Player in white jersey driving towards the basket past a defender in a teal jersey
Player in a white jersey performing a crossover move to get past a defender in a blue jersey near the three-point line, crowd in background
Player in a white jersey dunking the ball while a player in a turquoise jersey lies on the ground under the rim, and the crowd and bench react energetically


Just a short introduction to process images using gpt-4o, still a bad result, definitely need to change the prompt